# Agent with parse context and structures retrieval

https://docs.llamaindex.ai/en/stable/examples/finetuning/react_agent/react_agent_finetune/

https://docs.llamaindex.ai/en/stable/examples/agent/react_agent_with_query_engine/


In [2]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.llms.openai import OpenAI

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [3]:
llm_4 = OpenAI(model="gpt-4o-mini")

In [4]:
def create_and_load_index():
    persist_dir = "./storage/"
    try:
        storage_context = StorageContext.from_defaults(
            persist_dir=persist_dir,
        )
        tu_vi_index = load_index_from_storage(storage_context)
        index_loaded = True
    except:
        index_loaded = False
    
    if not index_loaded:
        docs = SimpleDirectoryReader(
            input_dir="../data/",
        ).load_data()
        tu_vi_index = VectorStoreIndex.from_documents(docs)
        tu_vi_index.storage_context.persist(persist_dir)

    return tu_vi_index

In [5]:
tu_vi_index = create_and_load_index()

## Hàm luận từ ngày tháng năm sinh.

In [6]:
from datetime import datetime

def calculate_supplementary_stars(year_stem, month_stem, day_stem, hour_stem):
    supplementary = {}

    # Hóa Kỵ: Phân tích theo các tương tác giữa các Can và Chi
    if (year_stem - day_stem) % 10 in [2, 4, 6]:
        supplementary['Hóa Kỵ'] = "Có thể gặp trở ngại trong công việc và cuộc sống."
    else:
        supplementary['Hóa Kỵ'] = "Tình hình ổn định, ít gặp trở ngại."

    # Hóa Lộc: Phân tích theo Can và Chi
    if (year_stem - month_stem) % 10 in [0, 2, 3]:
        supplementary['Hóa Lộc'] = "Tài lộc dồi dào, có cơ hội đầu tư."
    else:
        supplementary['Hóa Lộc'] = "Cần thận trọng trong tài chính."

    # Hóa Quyền: Xem xét khả năng lãnh đạo và ảnh hưởng
    if (day_stem - month_stem) % 10 in [3, 5, 7]:
        supplementary['Hóa Quyền'] = "Có khả năng lãnh đạo và ảnh hưởng đến người khác."
    else:
        supplementary['Hóa Quyền'] = "Không có khả năng lãnh đạo mạnh."

    # Hóa Sát: Xem xét có thể gặp khó khăn, thử thách
    if (hour_stem - year_stem) % 10 in [1, 4, 5]:
        supplementary['Hóa Sát'] = "Có thể gặp phải khó khăn hoặc trở ngại lớn."
    else:
        supplementary['Hóa Sát'] = "Cuộc sống trôi chảy, ít gặp khó khăn."

    return supplementary

def an_sao_tuvi(day, month, year, hour):
    # Thông tin cơ bản từ ngày tháng năm sinh
    can = ['Giáp', 'Ất', 'Bính', 'Đinh', 'Mậu', 'Kỷ', 'Canh', 'Tân', 'Nhâm', 'Quý']
    chi = ['Tý', 'Sửu', 'Dần', 'Mão', 'Thìn', 'Tỵ', 'Ngọ', 'Mùi', 'Thân', 'Dậu', 'Tuất', 'Hợi']
    cung_names = ['Mệnh', 'Phụ Mẫu', 'Phúc Đức', 'Điền Trạch', 'Quan Lộc', 'Nô Bộc', 
                  'Thiên Di', 'Tật Ách', 'Tài Bạch', 'Tử Tức', 'Phu Thê', 'Huynh Đệ']
    
    # Xác định Thiên Can và Địa Chi từ năm sinh
    can_index = (year % 10)
    chi_index = (year % 12)
    thien_can = can[can_index]
    dia_chi = chi[chi_index]
    
    # sao_phu = calculate_supplementary_stars(can_index, chi_index, day, hour)
    # print('sao phu', sao_phu)

    # Tính Cục dựa vào năm sinh
    cuc_mapping = {
        'Kim': ['Canh', 'Tân'],
        'Mộc': ['Giáp', 'Ất'],
        'Thủy': ['Nhâm', 'Quý'],
        'Hỏa': ['Bính', 'Đinh'],
        'Thổ': ['Mậu', 'Kỷ']
    }
    cuc = None
    for key, values in cuc_mapping.items():
        if thien_can in values:
            cuc = key
            break

    # An các cung dựa vào giờ sinh
    gio_mapping = {
        'Tý': 0, 'Sửu': 1, 'Dần': 2, 'Mão': 3,
        'Thìn': 4, 'Tỵ': 5, 'Ngọ': 6, 'Mùi': 7,
        'Thân': 8, 'Dậu': 9, 'Tuất': 10, 'Hợi': 11
    }
    gio_sinh = list(gio_mapping.keys())[hour % 12]
    
    # Bản đồ chi đến các cung trong lá số
    cung_chi_mapping = {
        'Tý': 'Mệnh', 'Sửu': 'Phụ Mẫu', 'Dần': 'Phúc Đức', 'Mão': 'Điền Trạch',
        'Thìn': 'Quan Lộc', 'Tỵ': 'Nô Bộc', 'Ngọ': 'Thiên Di', 'Mùi': 'Tật Ách',
        'Thân': 'Tài Bạch', 'Dậu': 'Tử Tức', 'Tuất': 'Phu Thê', 'Hợi': 'Huynh Đệ'
    }
    
    # An các sao chính tinh
    chinh_tinh = {
        'Tử Vi': 'Thìn', 'Thiên Phủ': 'Tuất', 'Thái Dương': 'Dần', 'Thái Âm': 'Hợi',
        'Thiên Cơ': 'Sửu', 'Thiên Lương': 'Ngọ', 'Vũ Khúc': 'Thân', 'Thiên Đồng': 'Tý',
        'Cự Môn': 'Dậu', 'Liêm Trinh': 'Mão', 'Thất Sát': 'Mùi', 'Phá Quân': 'Tỵ',
        'Tham Lang': 'Dậu'
    }
    
    # Phụ tinh cơ bản
    phu_tinh = {
        'Hóa Khoa': 'Mệnh', 'Hóa Quyền': 'Quan Lộc', 'Hóa Lộc': 'Tài Bạch', 'Hóa Kỵ': 'Phu Thê'
    }
    
    # Kết quả an sao
    sao_cung = {cung: [] for cung in cung_names}
    
    # An các chính tinh vào các cung
    for sao, chi_name in chinh_tinh.items():
        cung = cung_chi_mapping[chi_name]  # Lấy tên cung từ chi
        # print('an_sao_tuvi:chinh_tinh', cung, sao, chi_name)
        sao_cung[cung].append(sao)
    
    # An các phụ tinh vào các cung
    for sao, cung in phu_tinh.items():
        sao_cung[cung].append(sao)
    
    response = ""
    response += f"Lá số cho {day}/{month}/{year} giờ {gio_sinh}\n"
    response += f"Thiên Can: {thien_can}, Địa Chi: {dia_chi}, Cục: {cuc}\n"
    for cung, sao_list in sao_cung.items():
        response += f"Cung {cung}: các sao : {', '.join(sao_list) if sao_list else 'Không có sao'}\n"

    return response

an_sao_tuvi(1, 10, 1996, 4)

'Lá số cho 1/10/1996 giờ Thìn\nThiên Can: Canh, Địa Chi: Thìn, Cục: Kim\nCung Mệnh: các sao : Thiên Đồng, Hóa Khoa\nCung Phụ Mẫu: các sao : Thiên Cơ\nCung Phúc Đức: các sao : Thái Dương\nCung Điền Trạch: các sao : Liêm Trinh\nCung Quan Lộc: các sao : Tử Vi, Hóa Quyền\nCung Nô Bộc: các sao : Phá Quân\nCung Thiên Di: các sao : Thiên Lương\nCung Tật Ách: các sao : Thất Sát\nCung Tài Bạch: các sao : Vũ Khúc, Hóa Lộc\nCung Tử Tức: các sao : Cự Môn, Tham Lang\nCung Phu Thê: các sao : Thiên Phủ, Hóa Kỵ\nCung Huynh Đệ: các sao : Thái Âm\n'

In [7]:
from llama_index.core.tools import BaseTool, FunctionTool
from lunarcalendar import Converter, Solar

def convert_to_lunar(year, month, day):
    solar = Solar(year, month, day)
    lunar = Converter.Solar2Lunar(solar)
    return lunar

def fn_an_sao(birthday: str, hour: str):
    """Trích xuất birthday[date + hour] từ text người dùng nhập và gán sao theo các [cung mệnh, chính tinh và phụ tinh]"""
    print('get_birthday', birthday, hour)
    date_obj = datetime.strptime(birthday, "%d/%m/%Y")
    hour = hour.split(':')[0]
    hour_int = int(hour)

    lunar_date = convert_to_lunar(date_obj.year, date_obj.month, date_obj.day)

    results = an_sao_tuvi(lunar_date.day, lunar_date.month, lunar_date.year, hour_int)

    return results


In [8]:
from llama_index.core.agent import ReActAgent
from llama_index.agent.openai import OpenAIAssistantAgent
from llama_index.core.output_parsers import PydanticOutputParser
from enum import Enum
from pydantic import BaseModel, Field
from llama_index.agent.openai import OpenAIAgent
from pydantic import BaseModel
from typing import List
from llama_index.core.program import FunctionCallingProgram


prompt_template_str = """\
Extract info birthday(date + hour) from query:
{query}
"""

class User(BaseModel):
    """Data model for user info."""
    birthday: str
    hour: str
    name: str


programInfoUser = FunctionCallingProgram.from_defaults(
    output_cls=User,
    prompt_template_str=prompt_template_str,
    verbose=True,
)

# [Optional] Add Context
context = """\
Bạn là một ông thầy chuyên bói toán tử vi.\
bạn sẽ trả lời tư vấn cho người dùng về công danh, sự nghiệp, tình cảm, gia đình trong năm đó
"""

class CungMenh(Enum):
    MENH = "Mệnh"
    PHU_MAU = "Phụ Mẫu"
    PHUC_DUC = "Phúc Đức"
    DIEN_TRACH = "Điền Trạch"
    QUAN_LOC = "Quan Lộc"
    NO_BOC = "Nô Bộc"
    THIEN_DI = "Thiên Di"
    TAT_ACH = "Tật Ách"
    TAI_BACH = "Tài Bạch"
    TU_TUC = "Tử Tức"
    PHU_THE = "Phu Thê"
    HUYNH_DE = "Huynh Đệ"

class VanMenhTheoCung(BaseModel):
    """Data model vận mệnh theo cung và các diễn giải theo cung"""

    cung: CungMenh
    summary: str = Field(
        description="Mô tả chung về cung",
    )
    diengiai: str = Field(
        description="Diễn giải vận mệnh theo cung",
    )
    
class Tuvi(BaseModel):
    """Data model lá bài tử vi."""

    name: str
    birthday: str = Field(
        description="Ngày/Tháng/Năm sinh",
    )
    hour: str = Field(
        description="Giờ sinh",
    )
    vanmenh: List[VanMenhTheoCung]


In [9]:
message_chat = """
tôi tên Nguyễn Thành Hậu với ngày sinh 25/12/1999 13:00 thì tháng 6 năm 2025 sẽ như thế nào theo lá số tử vi ?
"""

programInfoUser_response = programInfoUser(query=message_chat)
programInfoUser_response.birthday, programInfoUser_response.hour
an_sao_res = fn_an_sao(programInfoUser_response.birthday, programInfoUser_response.hour)

query_engine = tu_vi_index.as_query_engine(
    output_cls=Tuvi, response_mode="tree_summarize", llm=llm_4, context=context
)

query_engine_response = query_engine.query(message_chat + an_sao_res)
print(str(query_engine_response))

=== Calling Function ===
Calling function: User with args: {"birthday": "25/12/1999", "hour": "13:00", "name": "Nguy\u1ec5n Th\u00e0nh H\u1eadu"}
=== Function Output ===
birthday='25/12/1999' hour='13:00' name='Nguyễn Thành Hậu'
get_birthday 25/12/1999 13:00
{"name":"Nguyễn Thành Hậu","birthday":"25/12/1999","hour":"13:00","vanmenh":[{"cung":"Mệnh","summary":"Cung Mệnh có các sao Thiên Đồng, Hóa Khoa","diengiai":"Cung Mệnh này mang lại sự thông minh, khéo léo và khả năng giải quyết vấn đề tốt. Hóa Khoa giúp hóa giải tai họa, mang lại sự hanh thông trong cuộc sống."},{"cung":"Phụ Mẫu","summary":"Cung Phụ Mẫu có sao Thiên Cơ","diengiai":"Sao Thiên Cơ trong cung này cho thấy sự hỗ trợ từ gia đình, đặc biệt là từ cha mẹ, giúp đỡ trong những thời điểm khó khăn."},{"cung":"Phúc Đức","summary":"Cung Phúc Đức có sao Thái Dương","diengiai":"Sao Thái Dương mang lại phúc lộc, tài vận tốt, có khả năng thu hút sự chú ý và tôn trọng từ người khác."},{"cung":"Điền Trạch","summary":"Cung Điền Trạch có